# LIME Text Explainer via XAI

This tutorial demonstrates how to generate explanations using LIME's text explainer implemented by the XAI library. Mucb of the tutorial overlaps with what is covered in the [LIME tabular tutorial](lime_tabular_explainer.ipynb). To recap, the main steps for generating explanations are:

1. Instantiate the Explainer class
2. Build the text explainer
3. Call `explain_instance`

### Step 1: Import libraries

In [1]:
# Some auxiliary imports for the tutorial
import sys
import random
import numpy as np
from pprint import pprint
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

# Set seed for reproducibility
np.random.seed(123456)

# Set the path so that we can import the Explainer
sys.path.append('../../../')

# Main XAI imports
import xai
from xai.explainer import Explainer

/Users/i330688/venv_xai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/i330688/venv_xai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Step 2: Load dataset and train a model

In this tutorial, we rely on the 20newsgroups text dataset, which can be loaded via sklearn's dataset utility. Documentation on the dataset itself can be found [here](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html). To keep things simple, we will extract data for 3 topics - baseball, Christianity, and medicine.

Our target model is a multinomial Naive Bayes classifier, which we train using TF-IDF vectors.

In [2]:
# Train on a subset of categories

categories = [
    'rec.sport.baseball',
    'soc.religion.christian',
    'sci.med'
]

raw_train = datasets.fetch_20newsgroups(subset='train', categories=categories)
print(list(raw_train.keys()))
print(raw_train.target_names)
print(raw_train.target[:10])
raw_test = datasets.fetch_20newsgroups(subset='test', categories=categories)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(raw_train.data)
y_train = raw_train.target

X_test = vectorizer.transform(raw_test.data)
y_test = raw_test.target

clf = MultinomialNB(alpha=0.1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

['data', 'filenames', 'target_names', 'target', 'DESCR', 'description']
['rec.sport.baseball', 'sci.med', 'soc.religion.christian']
[1 0 2 2 0 2 0 0 0 1]


0.9689336691855583

### Step 3: Instantiate the explainer

Here, we will use the LIME Text Explainer.

In [3]:
explainer = Explainer(domain=xai.DOMAIN.TEXT)

### Step 4: Build the explainer

This initializes the underlying explainer object.

In [4]:
explainer.build_explainer()

### Step 5: Generate some explanations

We provide the `explain_instance` method with the raw text - LIME's text explainer algorithm will conduct its own preprocessing in order to generate interpretable representations of the data. Hence we must define a custom `predict_fn` which takes a raw piece of text, vectorizes it via a pre-trained TF-IDF vectorizer, and passes the vector into the trained Naive Bayes model to generate class probabilities. LIME uses `predict_fn` to query our Naive Bayes model in order to learn its behavior around the provided data instance.

In [9]:
def predict_fn(instance):
    vec = vectorizer.transform(instance)
    return clf.predict_proba(vec)

exp = explainer.explain_instance(
    predict_fn=predict_fn,
    labels=[0, 1, 2],
    instance=raw_test.data[0],
    num_features=10
)

print('Label', raw_train.target_names[raw_test.target[0]])
pprint(exp)

/Users/i330688/venv_xai/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Label rec.sport.baseball
{0: {'confidence': 0.9604247937223921,
     'explanation': [{'feature': 'Mattingly', 'score': 0.15948847368512384},
                     {'feature': 'njit', 'score': -0.058093742192877086},
                     {'feature': 'tesla', 'score': -0.05226657434448741},
                     {'feature': 'luriem', 'score': 0.041491210763013854},
                     {'feature': 'Yankee', 'score': 0.040511388496675634},
                     {'feature': 'Lurie', 'score': 0.03945369764822541},
                     {'feature': 'Allegheny', 'score': 0.038682356052868946},
                     {'feature': 'PLAYERS', 'score': 0.038581528968433275},
                     {'feature': 'Liberalizer', 'score': 0.03748895841638309},
                     {'feature': 'game', 'score': 0.03528869693865674}]},
 1: {'confidence': 0.015984823571617023,
     'explanation': [{'feature': 'Mattingly', 'score': -0.05353213121137752},
                     {'feature': 'game', 'score': -0.020368749

Just like with the LIME tabular explainer, the output of `explain_instance` is a JSON-compatible object where each class index maps to the target model's confidence and the corresponding explanations generated by LIME. For text, each feature is a token.

In [7]:
exp = explainer.explain_instance(
    predict_fn=predict_fn,
    labels=[0, 1, 2],
    instance=raw_test.data[7],
    num_features=5
)

print('Label', raw_train.target_names[raw_test.target[7]])
pprint(exp)

Label sci.med
{0: {'confidence': 0.006374625691451515,
     'explanation': [{'feature': 'pain', 'score': -0.02740261143993559},
                     {'feature': 'sr', 'score': 0.02617688083387586},
                     {'feature': 'ai', 'score': -0.02391983644002593},
                     {'feature': 'Covington', 'score': -0.020875042515066302},
                     {'feature': 'mcovingt', 'score': -0.020699977679627755}]},
 1: {'confidence': 0.8824748491424798,
     'explanation': [{'feature': 'hp', 'score': 0.06962985800565993},
                     {'feature': 'doctor', 'score': 0.067793107925725},
                     {'feature': 'pain', 'score': 0.06680102769302987},
                     {'feature': 'kidney', 'score': 0.05490790579203543},
                     {'feature': 'Kidney', 'score': 0.05326854053175153}]},
 2: {'confidence': 0.11115052516607107,
     'explanation': [{'feature': 'hp', 'score': -0.07999974792513226},
                     {'feature': 'doctor', 'score': -0.047

In [8]:
exp = explainer.explain_instance(
    predict_fn=predict_fn,
    labels=[0, 1, 2],
    instance=raw_test.data[9],
    num_features=5
)

print('Label', raw_train.target_names[raw_test.target[9]])
pprint(exp)

Label soc.religion.christian
{0: {'confidence': 6.798212345437472e-05,
     'explanation': [{'feature': 'Bible', 'score': -0.002350080976348548},
                     {'feature': 'Scripture', 'score': -0.001434457771521199},
                     {'feature': 'Heaven', 'score': -0.0013811963568868962},
                     {'feature': 'Sin', 'score': -0.00137237244087949},
                     {'feature': 'specific', 'score': -0.0013611914394935853}]},
 1: {'confidence': 0.00044272540371258136,
     'explanation': [{'feature': 'Bible', 'score': -0.007407412195931128},
                     {'feature': 'Scripture', 'score': -0.00365836775767881},
                     {'feature': 'Heaven', 'score': -0.003652181996607399},
                     {'feature': 'immoral', 'score': -0.003469502264458388},
                     {'feature': 'Sin', 'score': -0.003246609821338069}]},
 2: {'confidence': 0.9994892924728337,
     'explanation': [{'feature': 'Bible', 'score': 0.00973653997148663},
         

### Step 6: Save and load the explainer

Like with the LIME tabular explainer, we can save and load the explainer via `load_explainer` and `save_explainer` respectively.

In [10]:
# Save the explainer somewhere

explainer.save_explainer('artefacts/lime_text.pkl')

In [12]:
# Load the saved explainer in a new Explainer instance

new_explainer = Explainer(domain=xai.DOMAIN.TEXT, algorithm=xai.ALG.LIME)
new_explainer.load_explainer('artefacts/lime_text.pkl')

exp = new_explainer.explain_instance(
    predict_fn=predict_fn,
    labels=[0, 1, 2],
    instance=raw_test.data[9],
    num_features=5
)

print('Label', raw_train.target_names[raw_test.target[9]])
pprint(exp)

/Users/i330688/venv_xai/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Label soc.religion.christian
{0: {'confidence': 6.798212345437472e-05,
     'explanation': [{'feature': 'Bible', 'score': -0.002393317095822573},
                     {'feature': 'eternal', 'score': -0.0013269313594894295},
                     {'feature': 'Sin', 'score': -0.0013264176687192157},
                     {'feature': 'Scripture', 'score': -0.0013192950166291022},
                     {'feature': 'logical', 'score': -0.0012341366536975005}]},
 1: {'confidence': 0.00044272540371258136,
     'explanation': [{'feature': 'Bible', 'score': -0.007634815018552455},
                     {'feature': 'Scripture', 'score': -0.0034759122077137384},
                     {'feature': 'Sin', 'score': -0.0033190824276768936},
                     {'feature': 'immoral', 'score': -0.003251004838219847},
                     {'feature': 'Heaven', 'score': -0.003229818320004681}]},
 2: {'confidence': 0.9994892924728337,
     'explanation': [{'feature': 'Bible', 'score': 0.01007923054519195},
   